# Match segments


1. translate one segment so that the ends are over the first one
2. compute some sort of distance


## Setup


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from math import floor

import numpy as np

from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.config.types import EDGE_ENDS_TO_CORNER
from snap_fit.config.types import EdgePos
from snap_fit.config.types import SegmentShape
from snap_fit.image.process import estimate_affine_transform
from snap_fit.image.process import find_contours
from snap_fit.image.process import find_corners
from snap_fit.image.process import transform_contour
from snap_fit.image.segment import Segment
from snap_fit.image.segment_matcher import SegmentMatcher
from snap_fit.image.utils import draw_contour
from snap_fit.image.utils import draw_corners
from snap_fit.image.utils import show_image_mpl
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.piece import Piece
from snap_fit.puzzle.sheet import Sheet
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager

## Load new data


In [ ]:
# 1. Configure ArUco Board and Detector
# Using defaults which match the printed board used for 'data/oca'
board_config = ArucoBoardConfig(markers_x=5, markers_y=7)
detector_config = ArucoDetectorConfig(board=board_config)

# 2. Initialize SheetAruco helper
# crop_margin is automatically calculated from the detector configuration
sheet_aruco = SheetAruco(detector_config)

# 3. Define the loader function
# SheetAruco.load_sheet handles loading, rectification, and Sheet creation
aruco_loader = sheet_aruco.load_sheet

# 4. define base folder
paths = get_snap_fit_paths()
data_dir = paths.data_fol / "oca"
lg.info(f"Loading data from {data_dir}")

# 5. instantiate manager and load
manager = SheetManager()
manager.add_sheets(folder_path=data_dir, pattern="*.jpg", loader_func=aruco_loader)

# Verify Sheets
sheets = manager.get_sheets_ls()
print(f"Managed Sheets: {len(sheets)}")
for sheet in sheets:
    print(f" - {sheet.img_fp.name}: {len(sheet.pieces)} pieces")

# Verify Pieces
pieces = manager.get_pieces_ls()
print(f"Total Pieces: {len(pieces)}")


## Get pieces and segments


In [ ]:
p1_index = 1

s1_type = EdgePos.LEFT  # concave in
# s1_type = EdgePos.BOTTOM  # convex out
# s1_type = EdgePos.RIGHT  # flat edge

p1 = pieces[p1_index]
rprint(f"using {p1.name}")
seg1 = p1.contour.segments[s1_type]

In [ ]:
# in the class now
rprint(seg1.shape)

### Visualize the segments


In [ ]:
img_contour_seg = p1.img_bw.copy() // 10
for _ei, edge_name in enumerate(EdgePos):
    segment = p1.contour.segments[edge_name]
    points = segment.points
    color = 250 if edge_name == s1_type else 120
    draw_contour(img_contour_seg, points, color=color)
show_image_mpl(img_contour_seg, figsize=(5, 5))


## Compute shape


In [ ]:
# transform the contour on x axis
source = seg1.coords
target = np.array([[0, 0], [0, 500]])
transform_matrix = estimate_affine_transform(source, target)

# transform the points of the source segment
s1_points_transformed = transform_contour(seg1.points, transform_matrix)

In [ ]:
img_contour_seg = p1.img_bw.copy() // 10
color = 250
draw_contour(
    img_contour_seg,
    s1_points_transformed + 100,
    color=color,
)
show_image_mpl(img_contour_seg, figsize=(5, 5))

In [ ]:
s1_xs = s1_points_transformed[:, 0, 0]
# s1_ys

flat_th = 20
out_count = (s1_xs < -flat_th).sum()
in_count = (s1_xs > flat_th).sum()
rprint(out_count, in_count)

count_th = 5
is_out = bool(out_count > count_th)
is_in = bool(in_count > count_th)
rprint(f"{is_out=} {is_in=}")

match (is_out, is_in):
    case True, False:
        segment_shape = SegmentShape.OUT
    case False, True:
        segment_shape = SegmentShape.IN
    case False, False:
        segment_shape = SegmentShape.EDGE
    case True, True:
        segment_shape = SegmentShape.WEIRD

rprint(segment_shape)